In [3]:
import pandas as pd
from ml_model import RiskMultiRegressor, route_from_addresses

In [4]:
model = RiskMultiRegressor()
model.load("risk_model_trained_large.pkl")

c:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MultiOutputRegressor from version 1.6.1 when using version 1.5.1. This might

In [5]:
test_routes = [
    ("ул. Мира, 15, Екатеринбург", "ул. Тверская, 7, Москва", ["truck", "rail"]),
    ("ул. Ленина, 10, Новосибирск", "пр. Литейный, 20, Санкт-Петербург", ["truck", "air"]),
    ("ул. Пушкина, 12, Воронеж", "ул. Советская, 55, Ростов-на-Дону", ["truck", "rail"])
]

In [12]:
#!/usr/bin/env python3
"""
Тестирование модели RiskMultiRegressor на фиксированных маршрутах
Вход: адрес "От" и "До", виды перевозки заранее заданы
Выход: прогноз module_risk и геоданные
"""

import pandas as pd
from ml_model import RiskMultiRegressor, route_from_addresses

# -----------------------------
# 1) Загружаем обученную модель
# -----------------------------
model = RiskMultiRegressor()
model.load("risk_model_trained_large.pkl")
print("Модель загружена: risk_model_trained_large.pkl")

# -----------------------------
# 2) Список тестовых маршрутов
# -----------------------------
test_routes = [
    ("ул. Мира, 15, Екатеринбург", "ул. Тверская, 7, Москва", ["truck", "rail"]),
    ("ул. Ленина, 10, Новосибирск", "пр. Литейный, 20, Санкт-Петербург", ["truck", "air"]),
    ("ул. Пушкина, 12, Воронеж", "ул. Советская, 55, Ростов-на-Дону", ["truck", "rail"]),
    ("ул. Советская, 25, Казань", "ул. Красная, 10, Самара", ["truck", "rail"]),
]

# -----------------------------
# 3) Функция тестирования маршрута
# -----------------------------
def test_route(start_address, end_address, modules):
    print(f"\nТестируем маршрут: {start_address} -> {end_address} с модулями {modules}")

    route = route_from_addresses(start_address, end_address, modules=modules, segments_per_leg=3)
    
    # Формируем признаки для модели
    features_list = []
    for leg in route['legs']:
        seg0 = leg['segments'][0]
        features_list.append({
            'lat_start': seg0['coords'][0],
            'lon_start': seg0['coords'][1]
        })
    
    X_test = pd.DataFrame(features_list)
    
    # Предсказание модели
    Y_pred = model.predict(X_test)
    
    # Вывод результатов
    for leg, pred in zip(route['legs'], Y_pred.to_dict(orient='records')):
        print(f"  Модуль: {leg['module']}")
        for seg in leg['segments']:
            risk_value = pred.get(f"avg_module_risk_{leg['module']}", None)
            temp = seg["weather_raw"]["properties"]["timeseries"][0]["data"]["instant"]["details"].get("air_temperature")
            wind = seg["weather_raw"]["properties"]["timeseries"][0]["data"]["instant"]["details"].get("wind_speed")
            print(f"    Адрес: {seg['address']} — прогноз module_risk: {risk_value}, температура: {temp}°C, ветер: {wind} м/с")

# -----------------------------
# 4) Тест всех маршрутов
# -----------------------------
for start, end, modules in test_routes:
    test_route(start, end, modules)


Модель загружена: risk_model_trained_large.pkl

Тестируем маршрут: ул. Мира, 15, Екатеринбург -> ул. Тверская, 7, Москва с модулями ['truck', 'rail']


KeyError: 'legs'

KeyError: 'legs'